<a href="https://colab.research.google.com/github/minhaz1172/NLP/blob/main/Text_Classification(word2vec_andML).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd


# upload kaggle json

In [2]:
from google.colab import files
files.upload()  # Upload the downloaded kaggle.json file here

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json


# download and unzip

In [3]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip -d data


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  imdb-dataset-of-50k-movie-reviews.zip
replace data/IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [4]:
df=pd.read_csv('/content/data/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.shape

(50000, 2)

In [6]:
# check one review
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [7]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


# separate 1st 20k rows for my project

In [8]:
#df=df.iloc[:20000]
#df.shape

## check duplicated

In [9]:
df.duplicated().value_counts()

,count
False,49582
True,418


In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.duplicated().value_counts()

,count
False,49582


# Text Preprocessing

## lowercasing

In [12]:
df['review']=df['review'].str.lower()
df['review']

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production. <br /><br />the...
2,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is..."
...,...
49995,i thought this movie did a down right good job...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,i am a catholic taught in parochial elementary...
49998,i'm going to have to disagree with the previou...


## remove html tags

In [13]:
import re
def remove_html_tags(text):
  pattern=re.compile('<.*?>')
  return pattern.sub(r'',text)

df['review']=df['review'].apply(remove_html_tags)
df['review']

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production. the filming tec...
2,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is..."
...,...
49995,i thought this movie did a down right good job...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,i am a catholic taught in parochial elementary...
49998,i'm going to have to disagree with the previou...


In [14]:
df['review'][1] # cheking removed tags of html

'a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well done.'

## remove urls

In [15]:
import re
def remove_urls(text):
  url_pattern=re.compile(r'https?://\S+|www\.\S+')
  return url_pattern.sub(r'',text)

df['review']=df['review'].apply(remove_urls)
df['review']

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production. the filming tec...
2,i thought this was a wonderful way to spend ti...
3,basically there's a family where a little boy ...
4,"petter mattei's ""love in the time of money"" is..."
...,...
49995,i thought this movie did a down right good job...
49996,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,i am a catholic taught in parochial elementary...
49998,i'm going to have to disagree with the previou...


## remove punctuation

In [16]:
import string
def remove_punc2(text):
  tex=text.lower()
  text=text.translate(str.maketrans('','',string.punctuation))
  return text

df['review']=df['review'].apply(remove_punc2)
df['review']

,review
0,one of the other reviewers has mentioned that ...
1,a wonderful little production the filming tech...
2,i thought this was a wonderful way to spend ti...
3,basically theres a family where a little boy j...
4,petter matteis love in the time of money is a ...
...,...
49995,i thought this movie did a down right good job...
49996,bad plot bad dialogue bad acting idiotic direc...
49997,i am a catholic taught in parochial elementary...
49998,im going to have to disagree with the previous...


In [17]:
print(df['review'].iloc[0])      # first review as plain string
print(type(df['review'].iloc[0]))


one of the other reviewers has mentioned that after watching just 1 oz episode youll be hooked they are right as this is exactly what happened with methe first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the wordit is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to manyaryans muslims gangstas latinos christians italians irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awayi would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare forget pretty picture

## remove stop words

In [18]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Preprocessing sentiment columns by one hotencoder

In [19]:
from sklearn.preprocessing import OneHotEncoder
Oh=OneHotEncoder()
df['sentiment']=Oh.fit_transform(df[['sentiment']]).toarray()
df['sentiment']

,sentiment
0,0.0
1,0.0
2,0.0
3,1.0
4,0.0
...,...
49995,0.0
49996,1.0
49997,1.0
49998,1.0


In [20]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,0.0
1,a wonderful little production the filming tech...,0.0
2,i thought this was a wonderful way to spend ti...,0.0
3,basically theres a family where a little boy j...,1.0
4,petter matteis love in the time of money is a ...,0.0


## Apply Word2Vec

In [21]:
!pip install gensim

In [22]:
!pip install --upgrade pip
!pip install --upgrade gensim
!pip install --upgrade Cython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.2 MB/s  0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.12
    Uninstalling Cython-3.0.12:
      Successfully uninstalled Cython-3.0.12


In [28]:
import nltk
!rm -rf /root/nltk_data/tokenizers/punkt
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# import Word2vec gensim

In [36]:
import gensim
from gensim.utils import simple_preprocess

sentences = []
for review in df['review']:
    # Use simple_preprocess directly on the review
    sentences.append(simple_preprocess(str(review)))

print(sentences[:5])

[['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', 'oz', 'episode', 'youll', 'be', 'hooked', 'they', 'are', 'right', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'methe', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'brutality', 'and', 'unflinching', 'scenes', 'of', 'violence', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', 'trust', 'me', 'this', 'is', 'not', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'timid', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', 'sex', 'or', 'violence', 'its', 'is', 'hardcore', 'in', 'the', 'classic', 'use', 'of', 'the', 'wordit', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'nickname', 'given', 'to', 'the', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'it', 'focuses', 'mainly', 'on', 'emerald', 'city', 'an', 'experimental', 'section', 'of', 'the', 'prison', 'where', 'all', 'the', 'cells', 'have', 'glass', '

In [37]:
from os import cpu_count
model=gensim.models.Word2Vec(
    window=10,
    min_count=0,
    workers=cpu_count()
)

In [38]:
model.build_vocab(sentences)

In [39]:
model.train(sentences,total_examples=model.corpus_count,epochs=model.epochs)

(41382945, 53424760)

In [40]:
len(model.wv.index_to_key)

202759

# Apply Word2vec Vectorization

In [41]:
def document_vector(doc):
  # remove out of voabulary words
  doc=[word for word in doc.split() if word in model.wv.index_to_key]
  return np.mean(model.wv[doc],axis=0)

In [42]:
# cheking
document_vector(df['review'].iloc[0])

array([ 0.82702184, -0.45397007,  0.0425096 ,  1.1602417 ,  0.03990299,
        0.5744476 ,  0.2104475 ,  1.0539907 , -0.32583696,  0.48700735,
       -0.606725  ,  0.37219474, -0.02973661,  0.21586221, -0.7635592 ,
       -0.04047345, -0.39197046,  0.21489075, -0.1981643 , -0.329616  ,
       -0.29293296, -0.35126823,  0.1260006 ,  0.74899316,  0.51782626,
        0.18224478, -0.18585114,  0.19163811,  0.31798422,  0.30456048,
       -0.2947155 , -0.47562963, -0.73160166, -0.5502418 ,  0.55438435,
        0.04794507,  0.63086134,  0.43238947,  0.3540657 , -0.3175088 ,
       -0.3581278 ,  0.3787958 ,  0.7982109 ,  0.09819394, -0.29936814,
        0.14050697, -0.6644377 , -0.22696479,  0.18037653,  0.1337124 ,
       -0.11787517, -0.93280697, -1.5567213 ,  0.47576785,  0.02244381,
       -0.00246594, -0.06689032,  0.5087217 , -0.607066  ,  0.44516426,
       -0.1637044 ,  0.6706925 ,  0.09480675, -0.04283414, -0.860602  ,
        0.06620122,  0.10780984,  0.14082235,  0.20768827, -0.76

 # whole review column vector representation

## Tqdm is a Python library that provides fast, extensible progress bars for loops and iterables. It is commonly used to visualize the progress of long-running operations in Python scripts, offering a clear indication of completion status and estimated time remaining.

In [43]:
from tqdm import tqdm

In [44]:
reviews=[]
for doc in tqdm(df['review'].values):
  reviews.append(document_vector(doc))

100%|██████████| 49582/49582 [1:14:40<00:00, 11.07it/s]


In [45]:
reviews=np.array(reviews)
reviews[0]

array([ 0.82702184, -0.45397007,  0.0425096 ,  1.1602417 ,  0.03990299,
        0.5744476 ,  0.2104475 ,  1.0539907 , -0.32583696,  0.48700735,
       -0.606725  ,  0.37219474, -0.02973661,  0.21586221, -0.7635592 ,
       -0.04047345, -0.39197046,  0.21489075, -0.1981643 , -0.329616  ,
       -0.29293296, -0.35126823,  0.1260006 ,  0.74899316,  0.51782626,
        0.18224478, -0.18585114,  0.19163811,  0.31798422,  0.30456048,
       -0.2947155 , -0.47562963, -0.73160166, -0.5502418 ,  0.55438435,
        0.04794507,  0.63086134,  0.43238947,  0.3540657 , -0.3175088 ,
       -0.3581278 ,  0.3787958 ,  0.7982109 ,  0.09819394, -0.29936814,
        0.14050697, -0.6644377 , -0.22696479,  0.18037653,  0.1337124 ,
       -0.11787517, -0.93280697, -1.5567213 ,  0.47576785,  0.02244381,
       -0.00246594, -0.06689032,  0.5087217 , -0.607066  ,  0.44516426,
       -0.1637044 ,  0.6706925 ,  0.09480675, -0.04283414, -0.860602  ,
        0.06620122,  0.10780984,  0.14082235,  0.20768827, -0.76

In [46]:
reviews.shape

(49582, 100)

# split X and Y

In [47]:
X=df.drop(columns=['sentiment'],axis=1)
y=df['sentiment']

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train,X_test,y_train,y_test=train_test_split(reviews,y,test_size=0.2,random_state=42)

# Train by RandomForestClassifier

In [52]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred=rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8204093980034285